<a href="https://colab.research.google.com/github/Michael-Jimenez-C/Investigaci-n-de-operaciones/blob/main/TareaIO4/Modelo_de_transporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from IPython.display import Latex, display_latex

In [38]:
def latex(matriz):
  cad=r"\left[\begin{matrix}"
  for i in matriz:
    for j in i:
      cad+=str(j)+r"&"
    cad=cad[:-1]+r"\\"
  cad=cad[:-2]
  return cad+r"\end{matrix}\right]"
def mostrar(matriz):
  display_latex(Latex(latex(matriz)))

In [45]:
class ModeloTransporte:
  def asignar(Ma,oferta,demanda, i,j):
    if oferta[i]==0 and demanda[i]==0:
      return None
    #Se encarga de asignar la oferta, revisa si se puede colocar la oferta disponible o si no coloca la demanda disponible
    if oferta[i]<=demanda[j]: 
      Ma[i][j]=oferta[i]
    else:
    #Elimina la demanda columna oferta de sus vectores
      Ma[i][j]=demanda[j]
    #resta la oferta
    oferta[i]-=Ma[i][j]
    demanda[j]-=Ma[i][j]

  def eliminar(K,oferta,demanda):
    #se encarga de remplazar por un número muy alto los valores que ya no se puedan tomar de la matriz
    for i in range(len(oferta)):
      if(oferta[i]==0):
        #Asigna 'M' a toda la columna
        K[i,:]=np.ones(len(K[i,:]))*1e5
    for i in range(len(demanda)):
      if(demanda[i]==0):
        #Asigna 'M' a toda la fila
        K[:,i]=np.ones(len(K[:,i]))*1e5

  def calcular(C,M):
    #La suma de la diagonal de la matriz de resultados columna la matriz de costos es igual a calcular el costo
    S=np.array(C).T@np.array(M)
    sol=sum([S[i][i] for i in range(len(S))])
    return sol

  def norOeste(C,oferta,demanda):
    #Construye la matriz de resultados
    M=[[0 for i in C[0]] for i in C]
    #Recorre todos los valores de la matriz, y hace la asignación
    for i in range(len(C)):
      for j in range(len(C[0])):
        ModeloTransporte.asignar(M,oferta,demanda,i,j)
    return M,ModeloTransporte.calcular(C,M)
    
  def minimoCosto(C,oferta,demanda,M=None,K=None):
    #Hace una copia de la matriz de costos
    if M==None:
      K=np.array(C)
      M=[[0 for i in C[0]] for i in K]
    #Selecciona los mínimos de cada fila
    minimos=[np.min(i) for i in K]
    #Toma el argumento minimo del vector de minimos
    fila=np.argmin(minimos)
    #toma el minimo elemento que concuerde con la columna de minimos
    columna=[np.argmin(i) for i in K][fila]

    #Asigna y elimina los valores de la matriz
    ModeloTransporte.asignar(M,oferta,demanda,fila,columna)
    ModeloTransporte.eliminar(K,oferta,demanda)

    #Si la suma de demanda y oferta es cero, entonces se terminó el algoritmo
    if sum(demanda)+sum(oferta)!=0:
      #Cuando la suma no es igual a cero se velve a llamar al método
      ModeloTransporte.minimoCosto(C,oferta,demanda,M,K)
    
    return M,ModeloTransporte.calcular(C,M)
    

  def vogel(C,oferta,demanda,M=None,K=None):
    if M==None:
      M=[[0 for i in C[0]] for i in C]
      K=np.array(C)
    #Obtiene la resta de filas y columnas
    pf=[abs(np.sort(K[:,i])[0]-np.sort(K[:,i])[1]) for i in range(len(demanda))]
    pc=[abs(np.sort(K[i,:])[0]-np.sort(K[i,:])[1]) for i in range(len(oferta))]

    #Si el maximo se encuentra en la resta de filas, se busca el minimo en la columna, en caso contrario hace lo mismo pero con la columna
    if np.max(pf)>np.max(pc):
      fila,columna=np.argmin(K[:,np.argmax(pf)]),np.argmax(pf)
    elif np.max(pf)<=np.max(pc):
      fila,columna=np.argmax(pc),np.argmin(K[np.argmax(pc),:])
    #Asigna en la fila y columna encontrada
    ModeloTransporte.asignar(M,oferta,demanda,fila,columna)
    ModeloTransporte.eliminar(K,oferta,demanda)
    #Si la suma da cero el algoritmo termina y si no entonces se vuelve a enviar la matriz al algoritmo
    if sum(demanda)+sum(oferta)!=0:
      ModeloTransporte.vogel(C,oferta,demanda,M,K)
    return M,ModeloTransporte.calcular(C,M)

  def solve(C,oferta,demanda):
    m1,s1=ModeloTransporte.norOeste(np.array(C),np.array(oferta),np.array(demanda))
    m2,s2=ModeloTransporte.minimoCosto(np.array(C),np.array(oferta),np.array(demanda))
    m3,s3=ModeloTransporte.vogel(np.array(C),np.array(oferta),np.array(demanda))
    return [m1,m2,m3][np.argmin([s1,s2,s3])],np.min([s1,s2,s3])
  

In [46]:
C=[[10,2,20,11],
   [12,7,9,20],
   [4,14,16,18]]
oferta=[15,25,10]
demanda=[5,15,15,15]
m,s=ModeloTransporte.solve(C,oferta,demanda)
mostrar(m)
print(s)

475


In [42]:
C=[[800,1300,400,700],
   [1100,1400,600,100],
   [600,1200,800,900]]
oferta=[12,17,11]
demanda=[10,10,10,10]
m,s=ModeloTransporte.solve(C,oferta,demanda)
mostrar(m)
print(s)

24600


In [43]:
C=[[800,700,500,200],
   [500,200,100,300],
   [600,400,300,500]]
oferta=[60,80,40]
demanda=[40,60,20,60]
m,s=ModeloTransporte.solve(C,oferta,demanda)
mostrar(m)
print(s)

50000
